<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA-25/blob/main/lesson_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***


---

# Занятие №10
# XGBoost
**23 апреля 2025г.**

XGBoost (Extreme Gradient Boosting) — это мощная и эффективная библиотека машинного обучения, основанная на методе **градиентного бустинга**. Она используется для решения задач классификации, регрессии и ранжирования.  

**Градиентный бустинг** — это техника **машинного обучения** для построения прогностических моделей. Она относится к классу **ансамблевых методов**, где несколько «слабых» моделей (обычно деревьев решений) объединяются в одну «сильную» модель.

Основная идея — строить модели **последовательно**, каждая из которых пытается **исправить ошибки** предыдущих, используя **градиентный спуск** для минимизации функции потерь.

### Основные особенности XGBoost:
- **Градиентный бустинг** — это метод ансамблирования, при котором несколько слабых моделей (обычно деревьев решений) объединяются для формирования сильной модели.
- **Эффективность** — XGBoost оптимизирован по скорости и использует продвинутые техники, такие как параллельные вычисления и буферизацию деревьев.
- **Поддержка регуляризации** — встроенная L1 и L2 регуляризация помогает предотвращать переобучение.
- **Обработка пропущенных значений** — библиотека умеет сама "угадывать", куда направить пропущенные значения при обучении деревьев.
- **Гибкость** — поддерживает различные типы задач, множество гиперпараметров, и может работать как с небольшими, так и с очень большими наборами данных.

### Где используется:
- Соревнования по машинному обучению (например, Kaggle)
- Финансовый анализ
- Диагностика в медицине
- Предиктивное обслуживание оборудования
- Маркетинговая аналитика

Cписок популярных альтернатив XGBoost — градиентного бустинга на решающих деревьях

1. **LightGBM (Microsoft)**
   - Быстрее XGBoost на больших данных.
   - Использует histogram-based алгоритм и leaf-wise рост дерева.
   - Хорош для задач с большим количеством признаков и категориальными данными.

2. **CatBoost (Yandex)**
   - Хорошо работает с категориальными признаками без необходимости one-hot encoding.
   - Снижает переобучение.
   - Прост в использовании: меньше параметров для настройки.

3. **HistGradientBoosting (sklearn)**
   - Встроенная реализация бустинга с гистограммами в Scikit-learn (`sklearn.ensemble.HistGradientBoostingClassifier/Regressor`).
   - Быстрый и поддерживает natively missing values.
   - Хорошая альтернатива без установки сторонних библиотек.


In [ ]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, r2_score
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, clear_output

from tqdm import tqdm

## Многоклассовая классификация с XGBoost
### Загрузка и предобработка данных

In [ ]:
# import gdown
# import warnings
# warnings.filterwarnings('ignore')
# gdown.download('https://drive.google.com/uc?id=1j54o4pHTm3HvaYTEtv_i4hOJGy5yNeZZ', verify=False)

data = pd.read_parquet("./data_modes.gzip")

In [ ]:
data.head()

Признаки

In [ ]:
use_columns = ['GTA1.DBinPU.Alzzo', 'GTA1.DBinPU.Bo', 'GTA1.DBinPU.DlPkf',
               'GTA1.DBinPU.DlPtgft', 'GTA1.DBinPU.DlPvf', 'GTA1.DBinPU.fi',
               'GTA1.DBinPU.hmGTD', 'GTA1.DBinPU.hmTG', 'GTA1.DBinPU.P1mvhTG',
               'GTA1.DBinPU.Pk', 'GTA1.DBinPU.Pmvh', 'GTA1.DBinPU.PmvhMOGTD',
               'GTA1.DBinPU.PmvhMOTG', 'GTA1.DBinPU.PmvyhMOGTD',
               'GTA1.DBinPU.PmvyhMOTG', 'GTA1.DBinPU.Prazrjag_navhode',
               'GTA1.DBinPU.Ptgpd', 'GTA1.DBinPU.Ptgvh', 'GTA1.DBinPU.Pvh',
               'GTA1.DBinPU.Pvyhlg', 'GTA1.DBinPU.Qtg', 'GTA1.DBinPU.Tk',
               'GTA1.DBinPU.Tn', 'GTA1.DBinPU.Tt', 'GTA1.DBinPU.Tvh1',
               'GTA1.DBinPU.Pzad']

X = data.loc[:,use_columns]

Целевая переменная

In [ ]:
data['target'] = data[[
    'full_power_mode', 'partial_power_mode',
    'increas_power_mode', 'decreas_power_mode', 'start_up_mode',
    'shutdown_mode', 'stopped_state_mode']].idxmax(axis=1)
y = data.loc[:, ['target']]

In [ ]:
y.value_counts()

Разделение на тестовую и тренировочную выборки

In [ ]:
from sklearn.model_selection import train_test_split
# Разделяем с учетом дисбаланса классов
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

Балансировка данных

In [ ]:
!pip install imblearn

Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sampling_strategy = {
    "full_power_mode": 1000,
    "stopped_state_mode": 1000,
    "partial_power_mode": 1000
}
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

In [ ]:
y_train_resampled.value_counts()

In [ ]:
sampling_strategy = {
    "full_power_mode": 100,
    "stopped_state_mode": 100,
    "partial_power_mode": 100
}
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)

In [ ]:
y_test_resampled.value_counts()

Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train_resampled, y_train_resampled)

In [ ]:
y_train_resampled.value_counts()

XGBoost — это алгоритм на базе деревьев решений. А деревья решений и бустинг на их основе **не чувствительны к масштабу признаков** (в отличие от моделей типа линейной регрессии, SVM или нейронных сетей).

In [ ]:
y_train_resampled

LabelEncoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
y_train_resampled_encoded = encoder.fit_transform(y_train_resampled.values[:,0])
y_test_resampled_encoded = encoder.transform(y_test_resampled.values[:,0])

In [ ]:
y_test_resampled_encoded

In [ ]:
encoder.classes_

### Инициализация и обучение модели

In [ ]:
!pip install xgboost
import xgboost as xgb

`xgb.XGBClassifier` — это класс из библиотеки **XGBoost**, предназначенный для решения **задач классификации** с помощью алгоритма **градиентного бустинга на деревьях решений**.

- строит ансамбль деревьев решений (по умолчанию — деревья классификации),
- на каждом шаге добавляет дерево, которое исправляет ошибки предыдущих,
- минимизирует выбранную **функцию потерь (objective)**.

---

**Применяется для:**

- **Бинарной классификации** (например, "да/нет", "спам/не спам")
- **Многоклассовой классификации** (например, классификация цифр, видов растений и т.д.)

---

Параметр **`objective`** в XGBoost `XGBClassifier` определяет **функцию потерь**, которую оптимизирует модель, и напрямую зависит от типа задачи — классификация, регрессия, ранжирование и т.д.


---

**Для задач многоклассовой классификации:**

- `'multi:softmax'` — многоклассовая классификация, возвращает **номер класса** напрямую.  
  Требуется параметр `num_class`.
- `'multi:softprob'` — многоклассовая классификация, возвращает **вектор вероятностей по каждому классу**.  
  Также требует `num_class`.

---

**Описание остальных гиперпараметров:**

- **`num_class=7`**  
  Количество уникальных классов в задаче классификации. В данном случае — 7 классов.

- **`max_depth=6`**  
  Максимальная глубина каждого дерева.  
  Глубокие деревья могут моделировать сложные зависимости, но также повышают риск переобучения.

- **`learning_rate=0.1`**
  Скорость обучения, понижает вклад каждого дерева в итоговое решение.  
  Меньшие значения требуют больше деревьев (`n_estimators`) для хорошей сходимости.

- **`n_estimators=100`**  
  Количество деревьев (итераций бустинга).  
  Итоговая модель будет объединением этих 100 слабых моделей.

- **`subsample=0.8`**  
  Доля случайно выбранных объектов для обучения каждого дерева.  
  Значения <1.0 могут помочь в борьбе с переобучением.

- **`colsample_bytree=0.8`**  
  Доля случайно выбранных признаков, используемых при построении каждого дерева.  
  Также снижает риск переобучения, особенно при большом числе признаков.

- **`random_state=42`**  
  Фиксирует генератор случайных чисел, обеспечивая воспроизводимость результатов.


In [ ]:
classifier = xgb.XGBClassifier(
    objective='multi:softmax',
    # objective='multi:softprob',
    num_class=7,
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

classifier.fit(X_train_resampled, y_train_resampled_encoded)

In [ ]:
y_pred = classifier.predict(X_test_resampled)

In [ ]:
y_pred

In [ ]:
classifier.predict_proba(X_test_resampled)

In [ ]:
np.argmax(classifier.predict_proba(X_test_resampled), axis=1)
# np.max(classifier.predict_proba(X_test_resampled), axis=1)

### Анализ качества модели
Матрица неточностей

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

conf_mat = confusion_matrix(y_test_resampled_encoded, y_pred)
ConfusionMatrixDisplay(conf_mat, display_labels=encoder.classes_).plot()
plt.xticks(rotation=90)
plt.show()

In [ ]:
print(classification_report(y_test_resampled_encoded, y_pred, target_names=encoder.classes_))

### Подбор значений гиперпараметров
**RandomizedSearchCV**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Создание модели XGB
classifier =  xgb.XGBClassifier(
    objective='multi:softmax', num_class=7, random_state=42)

# Сетка гиперпараметров
param_grid = {
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Использование RandomizedSearchCV для подбора параметров
random_search = RandomizedSearchCV(
    estimator=classifier, param_distributions=param_grid, n_iter=10, cv=5,
    n_jobs=-1, verbose=2, random_state=42)

# Обучение модели с RandomizedSearchCV
random_search.fit(X_train_resampled, y_train_resampled_encoded)

# Вывод лучших параметров
print("Лучшие параметры:", random_search.best_params_)

In [ ]:
# Оценка модели на тестовых данных
best_classifier = random_search.best_estimator_
# print("Точность на тестовой выборке:", best_classifier.score(X_test_resampled, y_test_resampled_encoded))
y_pred = best_classifier.predict(X_test_resampled)

### Анализ качества модели

In [ ]:
conf_mat = confusion_matrix(y_test_resampled_encoded, y_pred)
ConfusionMatrixDisplay(conf_mat, display_labels=encoder.classes_).plot()
plt.xticks(rotation=90)
plt.show()

In [ ]:
print(classification_report(y_test_resampled_encoded, y_pred, target_names=encoder.classes_))

## Прогнозирование с XGBoost

In [ ]:
# import gdown
# import warnings
# warnings.filterwarnings('ignore')
# url = "https://drive.google.com/drive/folders/1RtrAevJUYSgTbp0YUztxEBB8_VcvjgGH?usp=drive_link"
# gdown.download_folder(url, quiet=True, verify=False)

In [ ]:
import glob
parquetFileList = glob.glob(f'./option_0/*.gzip')

In [ ]:
df_list = []

for file in tqdm(parquetFileList):
    df = pd.read_parquet(file)
    df_list.append(df)

data = pd.concat(df_list, axis=0).sort_index().ffill().drop_duplicates()
data = data.dropna()

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 6), constrained_layout=True)

ax1.plot(data['GTA1.DBinPU.P'].index, data['GTA1.DBinPU.P'].values,
         linestyle='none', marker='.', markersize=1);
ax1.set_ylabel("Мощность, кВт");
ax1.set_ylim([5700,6100]);
ax1.tick_params(labelbottom=False)

ax2.plot(data['GTA1.DBinPU.Qtg'].index, data['GTA1.DBinPU.Qtg'].values,
         linestyle='none', marker='.', markersize=1);
ax2.set_ylabel("Расход топлива, м3/ч");
ax2.set_ylim([1500,2500]);
ax2.tick_params(labelbottom=False)

ax3.plot(data['GTA1.DBinPU.Tt'].index, data['GTA1.DBinPU.Tt'].values,
         linestyle='none', marker='.', markersize=1);
ax3.set_ylabel("Температура\nух. газов, $^\\circ$С");
# ax3.set_ylim([300,600]);
ax3.tick_params(rotation=45);

### Предобработка данных

Проредим и отфильтруем данные

In [ ]:
df = pd.DataFrame(data['GTA1.DBinPU.Tt'])
df = df.resample('1h').mean()
df.columns = ['Tt']
df = df[df['Tt'] > 300]

In [ ]:
df.plot(style='.', figsize=(10, 2), ms=3, title="Температура ух. газов", legend=False)
plt.ylabel("$^\\circ$С")
plt.show()

**Лаги** — это **значения временного ряда на предыдущих временных шагах**, которые используются как признаки (фичи) для предсказания будущих значений.

Функция для генерации лагов

In [ ]:
def create_lag_features(df, column, lags):
    """
    Создает лаги признака column в количестве lags и возвращает новый DataFrame.
    """
    df_lagged = df.copy()
    for lag in range(1, lags + 1):
        df_lagged[f"{column}_lag_{lag}"] = df_lagged[column].shift(lag)

    return df_lagged

In [ ]:
n_lags = 24
df = create_lag_features(df, 'Tt', n_lags)

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

Создаем вектр признаков и вектор целевых значений

In [ ]:
y = df.loc[:, ['Tt']]
X = df.loc[:, [col for col in df.columns if 'lag' in col]]

Разделим на тренировочную и тестовую выборки (**временной порядок важно сохранить!**)

In [ ]:
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

### Инициализация и обучение модели

In [ ]:
pred_model = xgb.XGBRegressor()
pred_model.fit(X_train, y_train)

Предсказание

In [ ]:
y_pred = pred_model.predict(X_test)

baseline_pred = X_test.mean(axis=1).values

### Анализ качества модели

In [ ]:
# Оценка
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))

print("RMSE:", round(rmse, 3))
print("Baseline RMSE:", round(baseline_rmse, 3))

In [ ]:
# Визуализация
plt.plot(y_test.values, label='Истина')
plt.plot(y_pred, label='Прогноз xgb')
plt.plot(baseline_pred, label='Прогноз baseline')
plt.legend()
plt.title("Прогноз временного ряда XGBoost")
plt.show()

### Автономный авторегрессионный прогноз (одна точка за раз)
На вход модели подаются предсказания, а не истинные значения из данных

In [ ]:
start = X_test.index[0]
end = X_test.index[-1]
timestamps = pd.date_range(start=start, end=end, freq='1H')

In [ ]:
predictions = []
lags = X_test.iloc[0].values.tolist()  # первые лаги из начала теста

for _ in timestamps:
    # Прогноз
    y_pred = pred_model.predict(np.array([lags]))[0]
    predictions.append(y_pred)

    # Обновляем лаги: сдвигаем влево и добавляем новый прогноз
    lags = lags[1:] + [y_pred]

predictions = pd.DataFrame(predictions, index=timestamps, columns=['Tt'])

baseline_predictions = [X_test.iloc[0].mean() for _ in y_test.index]

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(timestamps, predictions['Tt'], label='Прогноз xgb')
plt.plot(df['Tt'].loc[start:].index, df['Tt'].loc[start:], linestyle='none', marker='.', markersize=5, label='Истина')
plt.plot(y_test.index, baseline_predictions, label='Прогноз baseline')

plt.ylabel("$^\\circ$С")
plt.title('Autoregressive прогноз XGBoost')
plt.xlabel('Индекс времени')
plt.legend()
plt.grid(True)
plt.tight_layout()

# plt.xlim([start, start + pd.Timedelta("4d")])

plt.show()

In [ ]:
# Общие индексы
common_index = y_test.index.intersection(predictions.index)

In [ ]:
rmse = np.sqrt(
    mean_squared_error(y_test.loc[common_index], predictions.loc[common_index]))

baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_predictions))

print("RMSE:", round(rmse, 3))
print("Baseline RMSE:", round(baseline_rmse, 3))

### Модель с множественным прогнозом (сразу n точек)

**sequence-to-sequence (seq2seq)** прогноз с XGBoost  
модель получает на вход `input_len` прошлых значений и предсказывает сразу `output_len` будущих

XGBoost **не поддерживает** напрямую многомерный выход, но можно сделать обучить `output_len` отдельных моделей, каждая из которых будет предсказывать своё значение


In [ ]:
def make_seq2seq_dataset(series, input_len, output_len):
    X, Y = [], []
    for i in range(len(series) - input_len - output_len + 1):
        X.append(series[i:i + input_len])
        Y.append(series[i + input_len:i + input_len + output_len])
    return np.array(X), np.array(Y)

In [ ]:
input_len = 20
output_len = 20
X, Y = make_seq2seq_dataset(df['Tt'], input_len=input_len, output_len=output_len)

In [ ]:
X.shape, Y.shape

Разделение на тестовую и тренировочную выборки

In [ ]:
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = Y[:train_size], Y[train_size:]

`MultiOutputRegressor`

Это **обёртка**, которая позволяет использовать любую обычную **регрессионную модель** (например, `XGBRegressor`, `RandomForestRegressor`, `LinearRegression`) для решения задач **многомерной регрессии** (multi-output regression).

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

model = MultiOutputRegressor(xgb.XGBRegressor())
model.fit(X_train, y_train)  # X_train: (n_samples, 20), y_train: (n_samples, 20)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
baseline_pred = np.array([np.ones(20)*x.mean() for x in X_test])

In [ ]:
plt.plot(range(20), y_test[0], label="Истинные значения")
plt.plot(range(20), y_pred[0], label="Прогноз xgb", linestyle="--")
plt.plot(range(20), baseline_pred[0], label='Прогноз baseline')
plt.legend()
plt.title("Прогноз на 20 шагов вперёд")
plt.show()

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))

print("RMSE:", round(rmse, 3))
print("Baseline RMSE:", round(baseline_rmse, 3))

### Автономный авторегрессионный прогноз с swq2seq моделью

In [ ]:
current_input = X_test[0]
predictions = []

# Сколько раз надо сделать предсказания
n_iters = (len(timestamps) - input_len) // output_len

In [ ]:
for _ in range(n_iters):
    # Предсказание следующих 20 значений
    pred = model.predict(np.array([current_input]))[0]
    predictions.extend(pred)


    # Обновляем вход: последние 20 предсказанных значений
    current_input = pred.tolist()

In [ ]:
predictions = pd.DataFrame(predictions, index=timestamps[input_len:len(predictions)+input_len], columns=['Tt'])

In [ ]:
# Общие индексы
common_index = df.index.intersection(predictions.index)

In [ ]:
baseline_predictions = [X_test[0].mean() for _ in common_index]

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(
    predictions.index, predictions['Tt'], label="Прогноз xgb")
plt.plot(
    df['Tt'].loc[start:].index, df['Tt'].loc[start:],
    linestyle='none', marker='.', markersize=5,
    label="Истинные значения")
plt.plot(
    predictions.loc[common_index].index, baseline_predictions,
    label='Прогноз baseline')

plt.ylabel("$^\\circ$С")
plt.title('Autoregressive прогноз XGBoost')
plt.xlabel('Индекс времени')
plt.legend()
plt.grid(True)
plt.tight_layout()

# plt.xlim([start, start + pd.Timedelta("4d")])

plt.show()

In [ ]:
rmse = np.sqrt(
    mean_squared_error(
        df.loc[common_index, ['Tt']], predictions.loc[common_index]))

baseline_rmse = np.sqrt(
    mean_squared_error(df.loc[common_index, ['Tt']], baseline_predictions))

print("RMSE:", round(rmse, 3))
print("Baseline RMSE:", round(baseline_rmse, 3))